<a href="https://colab.research.google.com/github/GJ-007-sage/deep-learning/blob/main/Assignment4_finetuning_24B1809.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

english to german

In [6]:
!pip install transformers datasets gradio


In [7]:
import gradio as gr
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
import torch


In [8]:
# Load WMT16 English-German dataset
dataset = load_dataset("wmt16", "de-en", split="train")

# Display a sample
print(dataset[0])


{'translation': {'de': 'Wiederaufnahme der Sitzungsperiode', 'en': 'Resumption of the session'}}


preprocess the data

In [9]:
from datasets import load_dataset

# Load the WMT16 English-German dataset and specify the split
dataset = load_dataset("wmt16", "de-en")
train_data = dataset["train"]

# Display a sample of the training data to understand its structure
print(train_data[0])


{'translation': {'de': 'Wiederaufnahme der Sitzungsperiode', 'en': 'Resumption of the session'}}


In [10]:
print(dataset)
print(dataset["train"][0])

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 4548885
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 2169
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2999
    })
})
{'translation': {'de': 'Wiederaufnahme der Sitzungsperiode', 'en': 'Resumption of the session'}}


In [11]:
def preprocess_data(batch):
    print(batch)

In [12]:
# Load model and tokenizer
model_name = "Helsinki-NLP/opus-mt-en-de"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Tokenization function

def preprocess_data(batch):
    # Extract lists of English and German sentences from the batch
    english_texts = [item["en"] for item in batch["translation"]]
    german_texts = [item["de"] for item in batch["translation"]]

    # Tokenize English texts (source)
    inputs = tokenizer(english_texts, padding="max_length", truncation=True, max_length=128)

    # Tokenize German texts (target) as labels
    with tokenizer.as_target_tokenizer():
        targets = tokenizer(german_texts, padding="max_length", truncation=True, max_length=128)

    # Add the target input_ids as labels
    inputs["labels"] = targets["input_ids"]
    return inputs


# Tokenize the dataset
tokenized_dataset = dataset["train"].map(preprocess_data, batched=True)

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/768k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/797k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/298M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

Map:   0%|          | 0/4548885 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


KeyboardInterrupt: 

define training arguments and data collector

In [ ]:
# Training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True
)

# Data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


Initialize Trainer and Fine-Tune the Model

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer
)

# Train the model
trainer.train()


Setup Gradio interface

In [ ]:
def translate_text(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    outputs = model.generate(inputs["input_ids"])
    translation = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return translation

# Set up Gradio interface
demo = gr.Interface(
    fn=translate_text,
    inputs="text",
    outputs="text",
    title="English to German Translator",
    description="Translate English sentences to German using a fine-tuned transformer model."
)

# Launch the Gradio app
demo.launch()
